In [1]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup

In [2]:
stock_id = "6446"
url = "https://goodinfo.tw/tw/StockDetail.asp?STOCK_ID=" + stock_id
#payloads = {"STOCK_ID":stock_id}
headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}

# 持股比例分析

In [60]:
url = "https://goodinfo.tw/tw/EquityDistributionClassHis.asp?STOCK_ID="+stock_id+"&DISPLAY_CAT=%E6%8C%81%E6%9C%89%E6%AF%94%E4%BE%8B%E5%8D%80%E9%96%93%E5%88%86%E7%B4%9A%E4%B8%80%E8%A6%BD%28%E7%B0%A1%E5%8C%96%29&CHT_CAT=MONTH"
resp = requests.request(method="POST",url=url,headers=headers)
resp.encoding = "utf-8"

equityDclass_table = pd.read_html(resp.text)

In [61]:
df = equityDclass_table[18].dropna(axis='columns')
df.columns = df.columns.droplevel()
df

,月份,統計日期,收盤,漲跌(元),漲跌(%),集保庫存(萬張),≦10張,＞10張≦50張,＞50張≦100張,＞100張≦200張,＞200張≦400張,＞400張≦800張,＞800張≦1千張,＞1千張
0,22M12,12/16,523,-7,-1.32,30.27,9.8,13,6.82,8.17,8.12,7.62,1.51,45
1,22M11,11/25,530,+76,+16.74,30.27,9.93,12.5,6.48,7.84,6.94,6.41,0.84,49
2,22M10,10/28,454,-29.5,-6.1,30.27,10.1,12.6,6.85,7.37,6.65,6.86,0.88,48.7
3,22M09,09/30,483.5,-97.5,-16.78,28.58,10.3,12.8,6.81,7.59,6.73,6.08,0.98,48.7
4,22M08,08/26,581,+12,+2.11,28.58,9.95,12.5,7.01,7.66,6.98,5.79,1.91,48.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,14M07,07/01,-,-,-,18.84,3.75,7.05,9.14,6.64,6.72,5.89,0.89,59.9
112,14M06,06/03,-,-,-,18.84,3.75,6.87,8.87,6.88,6.24,6.14,0.9,60.4
113,14M05,05/02,-,-,-,18.78,3.03,6.61,8.49,6.84,6.47,6.05,1.85,60.7
114,14M04,04/01,-,-,-,18.77,2.34,6.27,8.43,7.1,5.99,4.59,1.84,63.4


# 乖離

In [54]:
url = "https://goodinfo.tw/tw/ShowK_ChartFlow.asp?RPT_CAT=DR_3M&STOCK_ID="+stock_id+"&CHT_CAT=WEEK"
#,data=payloads
resp = requests.request(method="POST",url=url,headers=headers)
resp.encoding = "utf-8"

flow_table = pd.read_html(resp.text)

In [55]:
url = "https://goodinfo.tw/tw/ShowK_ChartFlow.asp?RPT_CAT=DR_1Y&STOCK_ID="+stock_id+"&CHT_CAT=WEEK"
resp = requests.request(method="POST",url=url,headers=headers)
resp.encoding = "utf-8"

flow_table2 = pd.read_html(resp.text)

In [56]:
#加入周K
url = "https://goodinfo.tw/tw/ShowK_Chart.asp?STOCK_ID="+stock_id+"&CHT_CAT2=WEEK&PRICE_ADJ=F"
#,data=payloads
resp = requests.request(method="POST",url=url,headers=headers)
resp.encoding = "utf-8"

K_table = pd.read_html(resp.text)

In [57]:
df = flow_table[17].dropna(axis='rows')
#df.drop("季均線乖離率換算價格",axis=1)
df.columns = df.columns.droplevel()
df = df.drop(['-40%','-30%','-20%','-10%','40%','30%','20%','10%'],axis=1)
df = df.rename(columns = {"均線":"季均線"})
df

,交易週別,收盤價,漲跌價,漲跌幅,目前乖離(%),季均線
0,22W52,523,-2,-0.38%,+4.35,501.2
1,22W51,525,-12,-2.23%,+3.84,505.6
2,22W50,537,+5,+0.94%,+5.16,510.7
3,22W49,532,+26,+5.14%,+3.9,512
4,22W48,506,-22,-4.17%,-1.53,513.9
...,...,...,...,...,...,...
166,20W04,114.5,-1.5,-1.29%,+4.28,109.8
167,20W03,116,+1,+0.87%,+5.65,109.8
168,20W02,115,+2.5,+2.22%,+4.85,109.7
169,20W01,112.5,+11.5,+11.4%,+3.04,109.2


In [58]:
df2 = flow_table2[17].dropna(axis='rows')
df2.columns = df2.columns.droplevel()
df = df.assign(年均線 = df2['均線']).dropna(axis='rows')
df = df.drop(df[df["交易週別"]=="交易週別"].index)
df.reset_index(inplace=False,drop=False)
df

,交易週別,收盤價,漲跌價,漲跌幅,目前乖離(%),季均線,年均線
0,22W52,523,-2,-0.38%,+4.35,501.2,427.9
1,22W51,525,-12,-2.23%,+3.84,505.6,424.3
2,22W50,537,+5,+0.94%,+5.16,510.7,421.1
3,22W49,532,+26,+5.14%,+3.9,512,415.5
4,22W48,506,-22,-4.17%,-1.53,513.9,409
...,...,...,...,...,...,...,...
166,20W04,114.5,-1.5,-1.29%,+4.28,109.8,128.7
167,20W03,116,+1,+0.87%,+5.65,109.8,129.8
168,20W02,115,+2.5,+2.22%,+4.85,109.7,131
169,20W01,112.5,+11.5,+11.4%,+3.04,109.2,132.1


In [59]:
#交易週別
dfk = K_table[25].dropna(axis='columns')
dfk.columns = dfk.columns.droplevel()
dfk = dfk.drop(dfk[dfk["交易週別"]=="交易週別"].index)
dfk.reset_index(inplace=False,drop=False)

df = df.assign(開盤 = dfk['開盤']).dropna(axis='rows')
df = df.assign(最高 = dfk['最高']).dropna(axis='rows')
df = df.assign(最低 = dfk['最低']).dropna(axis='rows')

df = df.rename(columns = {"開盤":"週開盤"})
df = df.rename(columns = {"收盤價":"週收盤"})
df = df.rename(columns = {"最高":"週最高"})
df = df.rename(columns = {"最低":"週最低"})

dfk.reset_index(inplace=False,drop=False)

df

,交易週別,週收盤,漲跌價,漲跌幅,目前乖離(%),季均線,年均線,週開盤,週最高,週最低
0,22W52,523,-2,-0.38%,+4.35,501.2,427.9,523,530,522
1,22W51,525,-12,-2.23%,+3.84,505.6,424.3,536,540,518
2,22W50,537,+5,+0.94%,+5.16,510.7,421.1,530,552,516
3,22W49,532,+26,+5.14%,+3.9,512,415.5,500,535,488.5
4,22W48,506,-22,-4.17%,-1.53,513.9,409,530,540,503
...,...,...,...,...,...,...,...,...,...,...
166,20W04,114.5,-1.5,-1.29%,+4.28,109.8,128.7,114.5,116,113.5
167,20W03,116,+1,+0.87%,+5.65,109.8,129.8,116,117.5,112.5
168,20W02,115,+2.5,+2.22%,+4.85,109.7,131,114.5,124,113.5
169,20W01,112.5,+11.5,+11.4%,+3.04,109.2,132.1,101.5,119.5,99.9
